In [267]:
# /// script
# requires-python = ">=3.11"
# dependencies = [
#     "importlib",
#     "pandas",
#     "udi-grammar-py",``
# ]
# ///

In [268]:
from udi_grammar_py import Chart, Op, rolling
import pandas as pd
import sys
import template_generation
import schema_generation
import template_expansion
import paraphraser
import importlib
import json
sys.path.append('.')

## Use python to generate template training data.

In [276]:
importlib.reload(template_generation)
df = template_generation.generate()
print(df.shape)
df.head()

(7, 5)


,query_template,constraints,spec_template,query_type,creation_method
0,"How many <E> are there, grouped by <F:N>?","[F.C * 2 < E.C, F.C < 4]","{""source"": {""name"": ""<E>"", ""source"": ""<E.url>""...",question,template
1,"How many <E> are there, grouped by <F:N>?","[F.C * 2 < E.C, F.C >= 4, F.C < 25]","{""source"": {""name"": ""<E>"", ""source"": ""<E.url>""...",question,template
2,Is there a correlation between <F1:Q> and <F2:Q>?,"[F1.C > 10, F2.C > 10]","{""source"": {""name"": ""<E>"", ""source"": ""<E.url>""...",question,template
3,What is the distribution of <F:Q>?,[E.C > 20],"{""source"": {""name"": ""<E>"", ""source"": ""<E.url>""...",question,template
4,What is the distribution of <F:Q>?,"[E.C < 20, E.C > 3]","{""source"": {""name"": ""<E>"", ""source"": ""<E.url>""...",question,template


## update data schema based on files in ./datasets folder

In [270]:
importlib.reload(schema_generation)
schema_generation.main()

## Contextualize the template training data by putting in real entity names and fields if they satisfy the constraints.

In [271]:
importlib.reload(template_expansion)
with open('./datasets/schema.json') as f:
    schema_list = json.load(f)
    df = template_expansion.expand(df, schema_list)

df.head()

,query_template,constraints,spec_template,query_type,creation_method,query_base,spec,dataset_schema
0,"How many <E> are there, grouped by <F:N>?","[F.C * 2 < E.C, F.C < 4]","{""source"": {""name"": ""<E>"", ""source"": ""<E.url>""...",question,template,"How many penguins are there, grouped by sex?","{""source"": {""name"": ""penguins"", ""source"": ""./d...",penguins
0,"How many <E> are there, grouped by <F:N>?","[F.C * 2 < E.C, F.C < 4]","{""source"": {""name"": ""<E>"", ""source"": ""<E.url>""...",question,template,"How many penguins are there, grouped by island?","{""source"": {""name"": ""penguins"", ""source"": ""./d...",penguins
0,"How many <E> are there, grouped by <F:N>?","[F.C * 2 < E.C, F.C < 4]","{""source"": {""name"": ""<E>"", ""source"": ""<E.url>""...",question,template,"How many penguins are there, grouped by species?","{""source"": {""name"": ""penguins"", ""source"": ""./d...",penguins
0,"How many <E> are there, grouped by <F:N>?","[F.C * 2 < E.C, F.C < 4]","{""source"": {""name"": ""<E>"", ""source"": ""<E.url>""...",question,template,"How many fake_donors are there, grouped by sex?","{""source"": {""name"": ""fake_donors"", ""source"": ""...",fakePortal
0,"How many <E> are there, grouped by <F:N>?","[F.C * 2 < E.C, F.C < 4]","{""source"": {""name"": ""<E>"", ""source"": ""<E.url>""...",question,template,"How many fake_samples are there, grouped by sa...","{""source"": {""name"": ""fake_samples"", ""source"": ...",fakePortal


## The paraphraser will use LLM frameork to paraphrase the query_base into several options


In [272]:
importlib.reload(paraphraser)
df = paraphraser.paraphrase(df)
df.head()


,query_template,constraints,spec_template,query_type,creation_method,query_base,spec,dataset_schema,query,expertise,formality
0,"How many <E> are there, grouped by <F:N>?","[F.C * 2 < E.C, F.C < 4]","{""source"": {""name"": ""<E>"", ""source"": ""<E.url>""...",question,template,"How many penguins are there, grouped by sex?","{""source"": {""name"": ""penguins"", ""source"": ""./d...",penguins,"How many penguins are there, grouped by sex?",-1,-1
0,"How many <E> are there, grouped by <F:N>?","[F.C * 2 < E.C, F.C < 4]","{""source"": {""name"": ""<E>"", ""source"": ""<E.url>""...",question,template,"How many penguins are there, grouped by island?","{""source"": {""name"": ""penguins"", ""source"": ""./d...",penguins,"How many penguins are there, grouped by island?",-1,-1
0,"How many <E> are there, grouped by <F:N>?","[F.C * 2 < E.C, F.C < 4]","{""source"": {""name"": ""<E>"", ""source"": ""<E.url>""...",question,template,"How many penguins are there, grouped by species?","{""source"": {""name"": ""penguins"", ""source"": ""./d...",penguins,"How many penguins are there, grouped by species?",-1,-1
0,"How many <E> are there, grouped by <F:N>?","[F.C * 2 < E.C, F.C < 4]","{""source"": {""name"": ""<E>"", ""source"": ""<E.url>""...",question,template,"How many fake_donors are there, grouped by sex?","{""source"": {""name"": ""fake_donors"", ""source"": ""...",fakePortal,"How many fake_donors are there, grouped by sex?",-1,-1
0,"How many <E> are there, grouped by <F:N>?","[F.C * 2 < E.C, F.C < 4]","{""source"": {""name"": ""<E>"", ""source"": ""<E.url>""...",question,template,"How many fake_samples are there, grouped by sa...","{""source"": {""name"": ""fake_samples"", ""source"": ...",fakePortal,"How many fake_samples are there, grouped by sa...",-1,-1


## Export as json object

In [273]:
df.to_json('./out/training_data.json', orient='records')

In [274]:
print(df.columns)

Index(['query_template', 'constraints', 'spec_template', 'query_type',
       'creation_method', 'query_base', 'spec', 'dataset_schema', 'query',
       'expertise', 'formality'],
      dtype='object')


In [275]:
print(df.shape)

(4704, 11)
